In [ ]:
import plotly.express as px
import pandas as pd
from datetime import date
import datetime
from dateutil import tz
import json
import os
import json



# global variables, the value is current file path
BASE_PATH=os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

In [ ]:
remove_list = ["README.md", ".gitignore"]
path = BASE_PATH + "/../COVID-19/csse_covid_19_data/csse_covid_19_time_series"

#Read detailed US Confirmed CSV data
us_confirmed_df = pd.read_csv(path + "/time_series_covid19_confirmed_US.csv")

# Observe data
us_confirmed_df.head()


In [ ]:
#Restructure US Confirmd using pd.melt

col_used = list(us_confirmed_df.columns)[11:] + ['Lat','Long_','Combined_Key','Province_State']
us_confirmed_gathered_df = pd.melt(us_confirmed_df[col_used],id_vars=['Lat','Long_','Combined_Key','Province_State'])
        
us_confirmed_gathered_df.head()

In [ ]:
def save_location_json(df, file_name):
    # Loop through global confirmed df, Save Long&Lat information for each Province/State
    # Loop through US confirmed df, Save Long&Lat information for each City 
    data_dict = {}
    if 'global' in file_name:
        for _ , row in df.iterrows():   
            data_dict[row['Province/State']] = [row['Long'], row['Lat']]
    elif 'US' in file_name:
        for _ , row in df.iterrows(): 
            data_dict[row['Combined_Key']] = [row['Long_'], row['Lat']]
    
    # Save data to local as json file
    with open(file_name + ".json", "w")as f:
        f.write(json.dumps(data_dict))

#Process US add df
save_location_json(us_confirmed_gathered_df, "US_location")

In [ ]:
#Generate US Plotly Bubble Map


headerList = list(us_confirmed_gathered_df.head().columns.values)

fig = px.scatter_geo(us_confirmed_gathered_df, lat="Lat",lon="Long_",color = "Province_State",
                     hover_name="Combined_Key", animation_frame="variable", size="value", projection = "albers usa")
fig.show()